# Interpolação Paramétrica de Margens Foliares com Splines

Folhas são um dos mais importantes órgãos estruturais e funcionais das plantas, e as modelar computacionalmente é importante para o estudo de diversos problemas, como: Análise morfométrica das plantas; Reconstrução 3D da cobertura formada por copas de árvores; Estimar quantidade de luz que atravessa copas de árvores; Simular distribuição de água e luz nas folhas; Modelar resíduos de pesticidas na superficie foliar; Análise de crescimento e resposta das plantas; Entre outras.

Existem diversas técnicas para a reconstrução de folhas (e plantas como um todo), porém aplicadas à reconstrução 3D para usos como jogos digitais, por exemplo. Embora sejam representações realistas, deixam a desejar no quesito acurácia. Para aplicações que demandam simulações de ambiente, por exemplo, são necessários modelos extremamente precisos, extraídos de fontes reais.

## Escopo

Este documento se motiva do artigo "[A Leaf Modeling and Multi-Scale Remeshing Method for Visual Computation via Hierarchical Parametric Vein and Margin Representation](https://www.frontiersin.org/journals/plant-science/articles/10.3389/fpls.2018.00783/full)", em que é aplicado a técnica de reconstrução de superfícies de plantas para a modelagem estrutural funcional delas. A abordagem utilizada pode ser descrita com os seguintes passos:

- Aquisição de dados: Dada uma folha real, técnicas como Scan 3D com pré-processamento são utilizadas;
- Extração de características: Selecionar de forma hierárquica pontos importantes que definem o esqueleto da planta, formado por veias e margens, conectando-as com B-Splines;
- Modelagem paramétrica de superfície: Criar uma superfície auxiliar utilizando NURBS definida pelas curvas geradas anteriormente;
- Reamostragem e geração de malha: Pontos importantes são extraídos das curvas e o modelo de superfície, gerando uma malha a partir dessa extração;
- Otimização da malha: Gerar diferentes resoluções para a malha, com o objetivo de equilibrar eficiência computacional e preservação de detalhes.

![img](https://www.frontiersin.org/files/Articles/276320/fpls-09-00783-HTML/image_m/fpls-09-00783-g001.jpg)

Aqui daremos foco às duas primeiras etapas, de uma forma simplificada:

- Aquisição de dados: Dada a imagem de uma folha real, aplicaremos algoritmos de visão computacional para extrair suas margens;
- Extração de características: Obtidos pontos principais da margem, resolveremos o problema de interpolação com uma curva B-Spline periódica (fechada e contínua).

Mesmo simplificado, teremos uma base de estudo para a resolução do problema de modelagem de folhas.

## Sobre o problema de interpolação de pontos com uma B-spline periódica

O problema de se interpolar pontos com uma B-spline períodica se resume a encontrar os coeficiêntes dos polinômios que definem a curva, de modo que ela passe pelos pontos dados além de ser fechada e contínua. Podemos representar isso como um sistema linear, onde as incógnitas são os coeficientes de dontrole da Spline.

Por ser periódica, sua continúidade e fechamendo fazem com que o sistema linear resultante seja de banda cíclica, ou seja, um sistema linear cuja matriz é predominantemente composta por diagonais principais e secundárias (como uma matriz tridiagonal), mas com elementos não nulos conectando o início e o fim das diagonais, formando um ciclo.

 Isso reflete a necessidade de garantir que a curva spline seja suave e contínua ao fechar sobre si mesma, conectando o último ponto ao primeiro. Um exemplo de sistema linear de banda cíclica (para 5 incógnitas) pode ser representado por:

$$
\begin{bmatrix}
a & b & 0 & 0 & c \\
c & a & b & 0 & 0 \\
0 & c & a & b & 0 \\
0 & 0 & c & a & b \\
b & 0 & 0 & c & a \\
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2 \\
x_3 \\
x_4 \\
x_5 \\
\end{bmatrix}
=
\begin{bmatrix}
b_1 \\
b_2 \\
b_3 \\
b_4 \\
b_5 \\
\end{bmatrix}
$$

Note que os elementos \( b \) e \( c \) conectam o início e o fim da matriz, formando o ciclo característico desse tipo de sistema.


# Aquisição de dados

# Interpolação de pontos

Dados os pontos de interesse da folha, gostaríamos de obter uma forma de representar sua borda de forma paramétrica (isto é, através de um parâmetro que varia em um certo intervalo). Essa abordagem é interessante poins define uma representação contínua, que podemos posteriormente evaluar numa quantidade de pontos necessária para o que gostaríamos de utilizar.

Para isso, utilizaremos B-splines, pois podemos obter representações precisas, que oferecem controle local sobre a curva e que garantem a continuidade entre segmentos.

## B-Splines

Essas curvas são definidas como combinações lineares de funções polinomiais por partes, e possuem três elementos principais:

- Conjunto de pontos de controle
- Grau polinomial
- Vetor de nós

B-Splines não necessariamente passam pelos pontos de controle, apenas as utilizam para definir sua forma.

Seja o vetor de nós $T = \{t_0, t_1, \dots, t_m\}$, onde $T$ é uma sequência não decrescente com $t_i \in [0, 1]$. Agora defina os pontos de controls $P_0, P_1, \dots, P_n$. Defina o grau como $p = m - n - 1$.

Agora defina a função base como:
$$
N_{i,0}(t) =
\begin{cases}
1 \quad \text{se } t_i \leq t \leq t_{i+1} \\
0 \quad \text{caso contrário}
\end{cases}
$$

$$
N_{i,j}(t) = \frac{t-t_i}{t_{i+j} - t_i} N_{i, j-1}(t) + \frac{t_{i+j+1} - t}{t_{i+j+1} - t_{j-1}} N_{i+1, j-1}(t)
$$

Onde $j = 1, 2, \dots, p$. Então a curva definida por
$$
C(t) = \sum_{i=0}^n P_i N_{i,p}(t)
$$
é uma B-spline.

## Interpolação com B-Splines

Suponha que haja $ n+1 $ pontos $D_0, D_1, \dots, D_n$. Queremos encontrar uma B-spline de grau $p$, onde $p \leq n$, definida por $n+1$ pontos de controle que passa por todos pos pontos na ordem que são definidos.

Esta B-spline possui $n+1$ pontos de controle desconhecidos. Já que o parâmetro $t_k$ corresponde ao ponto $D_k$, utilizando $t_k$ na equação $C(t) de uma B-spline, teremos o seguinte:
$$
D_k = C(t_k) = \sum_{i=0}^n N_{i, p}(t_k) P_i \quad 0 \leq k \leq n
$$

Podemos organizar os valores $N_{i, p}(t_k)$ em uma matriz $N$ de dimensões $(n+1) \times (n+1)$ onde a linha $k$ contém os valores $N_{0,p}(u), N_{1,p}(u), \dots, N_{i,p}(u)$ evaluados em $t_k$:
$$
\mathbf{N} =
\begin{bmatrix}
N_{0,p}(t_0) & N_{1,p}(t_0) & N_{2,p}(t_0) & \cdots & N_{n,p}(t_0) \\
N_{0,p}(t_1) & N_{1,p}(t_1) & N_{2,p}(t_1) & \cdots & N_{n,p}(t_1) \\
\vdots       & \vdots       & \vdots       & \ddots & \vdots       \\
N_{0,p}(t_n) & N_{1,p}(t_n) & N_{2,p}(t_n) & \cdots & N_{n,p}(t_n)
\end{bmatrix}
$$

Vamos também representar os vetores $D_k$ e $P_i$ em matrizes $D$ e $P$, onde $D_k$ é um vetor no espaço de dimensão $s$ e que aparece na linha $k$ da matriz $D$. De forma similar, $P_i$ é um vetor no espaço de dimensão $s$ que aparece na linha $i$ da matriz $P$. A dimensão $s$ é a dimensão onde a spline está sendo evaluada. Temos:
$$
\mathbf{D} =
\begin{bmatrix}
d_{01} & d_{02} & d_{03} & \cdots & d_{0s} \\
d_{11} & d_{12} & d_{13} & \cdots & d_{1s} \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
d_{n1} & d_{n2} & d_{n3} & \cdots & d_{ns}
\end{bmatrix}
\qquad
\mathbf{P} =
\begin{bmatrix}
p_{01} & p_{02} & p_{03} & \cdots & p_{0s} \\
p_{11} & p_{12} & p_{13} & \cdots & p_{1s} \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
p_{n1} & p_{n2} & p_{n3} & \cdots & p_{ns}
\end{bmatrix}
$$

Dessa forma, podemos verificar que podemos representar o problema de intepolação com a seguinte equação:
$$ D = N \cdot P$$

Como a matriz $D$ contém os pontos de entrada e a matriz $N$ é obtida evaluando as funções bases da B-spline nos parâmetros dados, tanto $D$ quanto $N$ são conhecidas, e a única matriz desconhecida é $P$. Se solucionarmos a equação para $P$, obteremos os pontos de controle da B-spline que interpola os pontos, consequentemente obtendo a curva desejada.

### Resolvendo $D = NP$

Resolveremos a equação coluna por coluna, obtendo assim um sistema linear. Sendo $d^i$ a i-ésima coluna de $D$, e $p^i$ a i-ésina coluna de $P$, teremos o seguinte sistema:
$$ d^i = N \cdot p^i$$

Resolvendo para $p^i$, teremos a i-ésima coluna de $P$. Fazendo isso para cada coluna, teremos a matriz $P$ completa e todos os pontos de controle serão computados.

## Obtendo uma B-spline fechada e contínua

No caso de uma B-spline fechada e contínua (ou seja, periódica), o sistema linear muda porque precisamos garantir que a curva:

Passe por todos os pontos dados,
Seja suave (continuidade de derivadas até ordem $k-1$),
E feche perfeitamente, ou seja, o início e o fim da curva coincidem e suas derivadas também.
Isso faz com que o sistema linear resultante seja cíclico (ou de banda cíclica): além das diagonais principais, aparecem elementos não nulos conectando o início e o fim da matriz, formando um ciclo. Ou seja, a matriz $N$ deixa de ser apenas tridiagonal (ou banda) e passa a ter elementos nas "pontas" conectando o primeiro e o último ponto.

Matematicamente, você pode representar o sistema assim: $$ D = N_{\text{per}} \cdot P $$ onde $N_{\text{per}}$ é a matriz das funções base com condições de periodicidade (ou seja, levando em conta que $N_{i,p}(t_0)$ e $N_{i,p}(t_n)$ estão conectados, assim como suas derivadas).

O exemplo dado na célula mostra uma matriz cíclica para 5 incógnitas: $$ \begin{bmatrix} a & b & 0 & 0 & c \ c & a & b & 0 & 0 \ 0 & c & a & b & 0 \ 0 & 0 & c & a & b \ b & 0 & 0 & c & a \ \end{bmatrix} \begin{bmatrix} x_1 \ x_2 \ x_3 \ x_4 \ x_5 \ \end{bmatrix}
\begin{bmatrix} b_1 \ b_2 \ b_3 \ b_4 \ b_5 \ \end{bmatrix} $$

Portanto, para representar seu sistema no caso fechado/periódico, basta montar a matriz $N$ considerando as condições de periodicidade (resultando em uma matriz cíclica), e resolver o sistema linear: $$ D = N_{\text{per}} \cdot P $$ onde $N_{\text{per}}$ tem elementos conectando o início e o fim da matriz, garantindo a continuidade e o fechamento da curva.
